In [14]:

from transformers import pipeline
from tqdm import tqdm
from PIL import Image
import os

# Save the model and feature extractor
#model.save_pretrained("./models/helmet_vit")
#feature_extractor.save_pretrained("./models/helmet_vit")

# Loading the model and feature extractor for inference
helmet_detector = pipeline("image-classification", model="./models/helmet_vit")

# If you are running inference over multiple images, here's how to show progress
image_paths = ["./Media/riders_1.jpg", "./Media/riders_2.jpg", "./Media/riders_3.jpg"]  # List your test images

# Show progress for inference
results = []
for img_path in tqdm(image_paths, desc="Processing Images", ncols=100):
    result = helmet_detector(img_path)
    results.append((img_path, result))

# Print the inference results
for img_path, result in results:
    print(f"Results for {img_path}: {result}")



Device set to use cpu
Processing Images: 100%|██████████████████████████████████████████████| 3/3 [00:01<00:00,  2.81it/s]

Results for ./Media/riders_1.jpg: [{'label': 'LABEL_1', 'score': 0.9951735138893127}, {'label': 'LABEL_0', 'score': 0.004826511722058058}]
Results for ./Media/riders_2.jpg: [{'label': 'LABEL_1', 'score': 0.9950814247131348}, {'label': 'LABEL_0', 'score': 0.004918545950204134}]
Results for ./Media/riders_3.jpg: [{'label': 'LABEL_1', 'score': 0.9950237274169922}, {'label': 'LABEL_0', 'score': 0.004976344760507345}]


In [1]:
from flask import Flask, request, jsonify, render_template
from transformers import pipeline
from PIL import Image
import os

app = Flask(__name__)

# Load pre-trained model
model_path = "./models/helmet_vit"
helmet_detector = pipeline("image-classification", model=model_path)

# Create upload folder
UPLOAD_FOLDER = "./Media"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files["file"]
    if file.filename == "":
        return jsonify({"error": "No file selected"}), 400

    # Save the uploaded image
    file_path = os.path.join(app.config["UPLOAD_FOLDER"], file.filename)
    file.save(file_path)

    # Load and predict
    image = Image.open(file_path).convert("RGB")
    result = helmet_detector(image)

    # Delete the file after prediction
    os.remove(file_path)

    return jsonify({"prediction": result})

if __name__ == "__main__":
    app.run(debug=True)


C:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


C:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
from flask import Flask, request, jsonify, render_template
from transformers import pipeline, ViTImageProcessor
from PIL import Image
import os

app = Flask(__name__)

# Load pre-trained model
model_path = "./models/helmet_vit"
helmet_detector = pipeline(
    "image-classification",
    model=model_path,
    feature_extractor=ViTImageProcessor.from_pretrained(model_path)
)

# Create upload folder
UPLOAD_FOLDER = "./Media"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files["file"]
    if file.filename == "":
        return jsonify({"error": "No file selected"}), 400

    # Save the uploaded image
    file_path = os.path.join(app.config["UPLOAD_FOLDER"], file.filename)
    file.save(file_path)

    try:
        # Load and predict
        image = Image.open(file_path).convert("RGB")
        result = helmet_detector(image)
    except Exception as e:
        return jsonify({"error": str(e)}), 500
    finally:
        # Delete the file after prediction
        os.remove(file_path)

    return jsonify({"prediction": result})

if __name__ == "__main__":
    app.run(debug=True)


Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1